# MODELO EN PRODUCCIÓN:

Se quiere saber si el precio del mercado intradiario del día D (D = today + 1) será inferior {1} o superior {0} al precio del mercado diario. Existe un gap en el que no se apuesta por ninguna de las dos opciones.

Para ello, se debe cubrir previamente la plantilla excel *Plantilla.xlsx* con los datos internos específicos y ejecutar este notebook.

La salida del modelo se guardará en *'Resultados/data&model/resultado/resultado.csv'*.

In [2]:
#Importamos librerías

import pandas as pd
import numpy as np
import datetime as dt
import pickle
import Funciones_Extraccion as datos

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
#import altair as alt


%matplotlib inline
plt.style.use('ggplot')

#from xgboost import XGBClassifier
#from xgboost import plot_importance

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE

from sklearn.metrics import classification_report, confusion_matrix, precision_score

In [3]:
pd.options.display.max_columns= None

### Parámetros

In [4]:
path_model='/Users/blancabengoa/Desktop/KSchool/TFM/Modelo4/'
path_plantilla='./data&model/'
path_salida='./data&model/resultado/'
columns=['PERIODO', 'WD', 'PRECIO_MD_ESP', 'P48_EOLICA', 'P48_DEMANDA',
       'POT_DISP_CARBON', 'CARBON_D-1', 'EMB_ANUALES', 'EMB_HIPERANUALES']

d={10008: 'P48_CARBON',
   475: 'POT_DISP_HULLA_ANT',  #Los tendremos que sumar para obtener MW disponibles Carbón
   476: 'POT_DISP_HULLA_SB'
  }

#Para los datos del día anterior (today)
fecha_ini_ant=dt.datetime.today().strftime('%Y-%m-%d') + ' 00:00:00'
fecha_fin_ant=dt.datetime.today().strftime('%Y-%m-%d') + ' 23:59:59'

#Para los datos del día de estudio (today + 1)
fecha_ini_prev=dt.datetime.strptime(fecha_ini_ant,'%Y-%m-%d %H:%M:%S') + dt.timedelta(days=1)
fecha_fin_prev=dt.datetime.strptime(fecha_fin_ant,'%Y-%m-%d %H:%M:%S') + dt.timedelta(days=1)

### Lectura datos externos - plantilla.###

Estos datos se tendrán que cumplimentar manualmente en la plantilla *Plantilla.xlsx*.

* Las previsiones de eólica, demanda las recibimos de un proveedor externo horariamente (en la realidad se podrá atacar a nuestro servidor a través de query insertándola en el notebook). Ejemplo de query:

```
  SELECT f.[FECHAVIENTO]
        ,f.[HORA]
        ,f.[VIENTO]
        ,f.[FECHAPREVISION]
  FROM (select [FECHAVIENTO], [HORA], max([FECHAPREVISION]) as maxprev
  from [BBDD]  -- representa la bbdd de nuestro servidor
  group by [FECHAVIENTO],[HORA]) as x
  inner join [BBDD] as f
  on f.[FECHAVIENTO]=x.[FECHAVIENTO] and f.[HORA]=x.[HORA] and f.[FECHAPREVISION]=maxprev 
  where f.[FECHAVIENTO]>= fecha_ini_prev 
  order by  f.[FECHAVIENTO],f.[HORA]

```
  
  
* El número de centrales arrancadas también las tenemos que descargar de nuestra bbdd interna a través de query.

```
  SELECT
      COUNT(DISTINCT(id_uprog)) AS NUM_PLANTAS
      ,format([FECHASESION],'yyyy-MM-dd') + ' ' + format([HORA]-1,'00') + ':00:00' as datetime
      FROM [BBDD]  -- representa la bbdd de nuestro servidor
      where id_uprog in ('','',...)  -- lista de unidades. Existen 26 centrales de carbón activas en España
	  and fechasesion=fecha_ini_ant
    GROUP BY datetime
    
```

* La previsión de precio la elabora otro departamento y lo facilita vía mail.

* La situación de los embalses se debe consultar en el Balance eléctrico diario REE y tomar el dato diario como horario. Link:

    https://www.ree.es/es/estadisticas-del-sistema-electrico-espanol/balance-diario

In [5]:
datos_externos=pd.read_excel(path_plantilla + 'Plantilla.xlsx')
datos_externos

,PRECIO_MD_ESP,P48_EOLICA,P48_DEMANDA,EMB_ANUALES,EMB_HIPERANUALES,NUM_PLANTAS_D-1
0,63.00,6211,29344,4618,3418,13
1,56.50,6052,27001,4618,3418,13
2,57.50,5794,25757,4618,3418,13
3,57.00,5436,25236,4618,3418,13
4,58.00,4956,25198,4618,3418,14
5,60.25,4375,25887,4618,3418,15
6,63.75,3763,28715,4618,3418,16
7,68.00,3272,33840,4618,3418,16
8,74.50,2885,37281,4618,3418,16
9,74.75,2429,38406,4618,3418,16


### Lectura datos esios

In [6]:
P48_CARBON=datos.solicita_datos_general(fecha_ini_ant,fecha_fin_ant,10008)  
P48_CARBON.columns=['P48_CARBON_D-1']
P48_CARBON.reset_index(inplace=True,drop=True)

POT_DISP_CARBON=datos.solicita_datos_general(fecha_ini_prev,fecha_fin_prev,475)+datos.solicita_datos_general(fecha_ini_prev,fecha_fin_prev,476)
POT_DISP_CARBON.columns=['POT_DISP_CARBON']
POT_DISP_CARBON.reset_index(inplace=True)

In [7]:
datos_esios=pd.DataFrame()
datos_esios=pd.concat([POT_DISP_CARBON,P48_CARBON],axis=1)
datos_esios

,datetime,POT_DISP_CARBON,P48_CARBON_D-1
0,2019-01-15 00:00:00,8656.4,4037.3
1,2019-01-15 01:00:00,8656.4,3433.4
2,2019-01-15 02:00:00,8656.4,3312.7
3,2019-01-15 03:00:00,8656.4,3385.0
4,2019-01-15 04:00:00,8656.4,3550.0
5,2019-01-15 05:00:00,8656.4,3574.0
6,2019-01-15 06:00:00,8656.4,4627.2
7,2019-01-15 07:00:00,8656.4,5223.5
8,2019-01-15 08:00:00,8656.4,5455.5
9,2019-01-15 09:00:00,8656.4,5597.2


### X final

Juntamos ambos datasets:

In [8]:
df=pd.DataFrame()
df=pd.concat([datos_esios,datos_externos],axis=1)
df.head()

,datetime,POT_DISP_CARBON,P48_CARBON_D-1,PRECIO_MD_ESP,P48_EOLICA,P48_DEMANDA,EMB_ANUALES,EMB_HIPERANUALES,NUM_PLANTAS_D-1
0,2019-01-15 00:00:00,8656.4,4037.3,63.0,6211,29344,4618,3418,13
1,2019-01-15 01:00:00,8656.4,3433.4,56.5,6052,27001,4618,3418,13
2,2019-01-15 02:00:00,8656.4,3312.7,57.5,5794,25757,4618,3418,13
3,2019-01-15 03:00:00,8656.4,3385.0,57.0,5436,25236,4618,3418,13
4,2019-01-15 04:00:00,8656.4,3550.0,58.0,4956,25198,4618,3418,14


Convertimos P48_CARBON_D-1 y NUM_CENTRALES_D-1 al feature CARBON_D-1

In [9]:
df['CARBON_D-1']=df['P48_CARBON_D-1']/df['NUM_PLANTAS_D-1']
df.head()

,datetime,POT_DISP_CARBON,P48_CARBON_D-1,PRECIO_MD_ESP,P48_EOLICA,P48_DEMANDA,EMB_ANUALES,EMB_HIPERANUALES,NUM_PLANTAS_D-1,CARBON_D-1
0,2019-01-15 00:00:00,8656.4,4037.3,63.0,6211,29344,4618,3418,13,310.561538
1,2019-01-15 01:00:00,8656.4,3433.4,56.5,6052,27001,4618,3418,13,264.107692
2,2019-01-15 02:00:00,8656.4,3312.7,57.5,5794,25757,4618,3418,13,254.823077
3,2019-01-15 03:00:00,8656.4,3385.0,57.0,5436,25236,4618,3418,13,260.384615
4,2019-01-15 04:00:00,8656.4,3550.0,58.0,4956,25198,4618,3418,14,253.571429


Features temporales: PERIODO, WD

In [10]:
df['datetime']=pd.to_datetime(df['datetime'],yearfirst=True,format='%Y-%m-%d %H:%M:%S')
df['PERIODO']=df['datetime'].dt.hour+1
df['WD']=pd.Series(df['datetime'].map(datos.WD))

In [11]:
X=df[columns]
X

,PERIODO,WD,PRECIO_MD_ESP,P48_EOLICA,P48_DEMANDA,POT_DISP_CARBON,CARBON_D-1,EMB_ANUALES,EMB_HIPERANUALES
0,1,2,63.00,6211,29344,8656.4,310.561538,4618,3418
1,2,2,56.50,6052,27001,8656.4,264.107692,4618,3418
2,3,2,57.50,5794,25757,8656.4,254.823077,4618,3418
3,4,2,57.00,5436,25236,8656.4,260.384615,4618,3418
4,5,2,58.00,4956,25198,8656.4,253.571429,4618,3418
5,6,2,60.25,4375,25887,8656.4,238.266667,4618,3418
6,7,2,63.75,3763,28715,8656.4,289.200000,4618,3418
7,8,2,68.00,3272,33840,8656.4,326.468750,4618,3418
8,9,2,74.50,2885,37281,8656.4,340.968750,4618,3418
9,10,2,74.75,2429,38406,8656.4,349.825000,4618,3418


### Clasificador: model

In [12]:
model=pickle.load(open(path_model + "clfxb_4.pkl","rb"))

In [13]:
y_pred=list(model.predict(X))
prob=list(model.predict_proba(X))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [14]:
df_pred_prob=pd.DataFrame(prob,columns=['PROB_0','PROB_1'])
df_pred_prob['PRED']=np.empty(len(df_pred_prob['PROB_0']))

for k,i in enumerate(y_pred):
    if (df_pred_prob['PROB_1'].iloc[k]>0.6) or (df_pred_prob['PROB_1'].iloc[k]<0.35):
        df_pred_prob['PRED'][k]=i
    else:
        df_pred_prob['PRED'][k]=-1
df_pred_prob

,PROB_0,PROB_1,PRED
0,0.582242,0.417758,-1.0
1,0.616493,0.383507,-1.0
2,0.695042,0.304958,0.0
3,0.647149,0.352851,-1.0
4,0.632950,0.367050,-1.0
5,0.637519,0.362481,-1.0
6,0.598452,0.401548,-1.0
7,0.611263,0.388737,-1.0
8,0.764974,0.235026,0.0
9,0.682927,0.317073,0.0


In [15]:
df_pred_prob.to_csv(path_salida + 'resultado_' + fecha_ini_prev.strftime('%Y-%m-%d') + '.csv',sep=';')